In [4]:
import torch
import torch.nn as nn
from torchsummary import summary
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import re
import itertools
import string
import random

In [2]:
%time
# Типа пайплайн первичных данных
path1 = "/Users/romanvisotsky/Downloads/data/main1.csv"
path2 = "/Users/romanvisotsky/Downloads/data/main2.csv"
path3 = "/Users/romanvisotsky/Downloads/data/main3.csv"

df = pd.read_csv(path1)
df_2 = pd.read_csv(path2)
df_3 = pd.read_csv(path3)

def clean_fio(fio):
 
    # Убираем цифры между буквами (без удаления пробелов)
    fio = re.sub(r'(\D)\d+(\D)', r'\1\2', fio)
    # Удаляем оставшиеся цифры (в начале и в конце)
    fio = re.sub(r'\d+', '', fio)
    fio = re.sub(r'[^\w\s]', '', fio) #Удаление всех символов, кроме букв (и цифр, но их мы уже удалили)
 
    fio = re.sub(r'\b[А-ЯA-Z]{2}[а-яa-z]+\b', '', fio) #Удаление всех слов где начало с 2х заглавных, а потом обычные
    fio = re.sub(r'\b[А-ЯA-Z][а-яa-z]+\b', '', fio) #Тоже самое с 1 заглавной
    fio = re.sub(r'\b[а-яa-z]+\b', '', fio) # Только строчные
 
    fio = re.sub(r'\b[оглиыуОГЛИЫУoO]+\b', '', fio, flags=re.IGNORECASE) #Удаляем угли, углы, огли, оглы (точнее любые слова состоящие только из сочетаний из этих букв)
 
    words = fio.split()
    unique_words = list(dict.fromkeys(words))  # Сохраняем порядок и удаляем дубликаты
    fio = ' '.join(unique_words)  # Обратно соединяем слова в строку
 
    # Убираем лишние пробелы
    fio = fio.strip()
    fio = re.sub(r'\s+', ' ', fio)
 
    fio = fio.upper()
    
    return fio
 
def clean_and_merge_fio(df):
    # Объединяем части ФИО и заполняем NaN пустыми строками
    df['full_name'] = df['last_name'].fillna('') + ' ' + df['first_name'].fillna('') + ' ' + df['middle_name'].fillna('')
 
    # Удаляем дублирующиеся буквы
    df['full_name'] = df['full_name'].str.replace(r'(.)\1+', r'\1', regex=True)
 
    # Удаление цифр и специальных символов
    df['full_name'] = df['full_name'].str.replace(r'(\D)\d+(\D)', r'\1\2', regex=True)
    df['full_name'] = df['full_name'].str.replace(r'\d+', '', regex=True)
    df['full_name'] = df['full_name'].str.replace(r'[^\w\s]', '', regex=True)
 
    # Удаляем слова, содержащие "нет" или "отсутствует"
    df['full_name'] = df['full_name'].apply(lambda x: ' '.join(word for word in x.split() if not re.search(r'\b(нет|отсутствует)\b', word, flags=re.IGNORECASE)))
 
    # Удаляем нежелательные слова
    df['full_name'] = df['full_name'].str.replace(r'\b[А-ЯA-Z]{2}[а-яa-z]+\b', '', regex=True)
    df['full_name'] = df['full_name'].str.replace(r'\b[А-ЯA-Z][а-яa-z]+\b', '', regex=True)
    df['full_name'] = df['full_name'].str.replace(r'\b[а-яa-z]+\b', '', regex=True)
    df['full_name'] = df['full_name'].str.replace(r'\b[оглиыуОГЛИЫУoO]+\b', '', regex=True, flags=re.IGNORECASE)
 
    # Удаление дубликатов и сохранение порядка
    df['full_name'] = df['full_name'].apply(lambda x: ' '.join(dict.fromkeys(x.split())))
 
    # Убираем лишние пробелы и приводим к верхнему регистру
    df['full_name'] = df['full_name'].str.replace(r'\s+', ' ', regex=True).str.strip().str.upper()
 
    # Заменяем пустые строки на NaN
    df['full_name'] = df['full_name'].replace('', np.nan)
 
    return df
 
 
def process_birthdates(birthdate_series):
    def correct_date(date_str):
        # Удаляем символы до первой цифры
        date_str = re.sub(r'^[^\d]*', '', date_str)
 
        # Проверяем год
        year_match = re.match(r'^(\d+)', date_str)
        if year_match:
            year_str = year_match.group(0)
            year = int(year_str)
            year_length = len(year_str)
 
            # Проверка на валидность года перед изменениями
            if 1900 <= year <= 2024:
                return date_str  # Год валиден, возвращаем строку
 
            if year_length > 4:
                if year_str.startswith('0'):
                    year_str = year_str[1:]  # Удаляем первую цифру, если 0
                elif year_str.startswith('10'):
                    year_str = year_str[0] + year_str[2:]  # Удаляем 0 после 1
 
                year = int(year_str)
 
            # Проверка на валидность года перед изменениями
                if 1900 <= year <= 2024:
                    date_str = date_str.replace(year_match.group(0), str(year), 1)
                    return date_str  # Год валиден, возвращаем строку
 
            if year_length == 3:
                year_str = '1' + year_str  # Добавляем 1 в начало
                year = int(year_str)
 
                # Проверка на валидность года
                if 1900 <= year <= 2024:
                    date_str = date_str.replace(year_match.group(0), str(year), 1)
                    return date_str  # Год валиден, возвращаем строку
 
            # Обработка 4-значного года
            if len(year_str) == 4:
                # Если год меньше 1900 и 2, 3 цифра равны 0, меняем первую цифру на 2
                if year < 1900 and len(str(year)) >= 3 and str(year)[1] == '0' and (str(year)[2] == '0' or str(year)[2] == '1'):
                    year_str = '2' + str(year)[1:]  # Изменение первой цифры
                    year = int(year_str)
 
                if year < 1900:
                    year_str = year_str[0] + '9' + year_str[2:]# Изменение первой цифры
                    year = int(year_str)
 
                # Если год больше 2024 и вторая цифра 9, изменяем первую цифру
                if year > 2024 and len(str(year)) >= 2 and str(year)[1] == '9':
                    year_str = '1' + str(year)[1:]  # Изменение первой цифры
                    year = int(year_str)
 
 
                # Проверяем валидность года после изменения
                if 1900 <= year <= 2024:
                    date_str = date_str.replace(year_match.group(0), str(year), 1)
                    return date_str
 
                # Если год не валиден, пробуем перестановки
                permutations = set(itertools.permutations(year_str))
                valid_years = [int(''.join(p)) for p in permutations if 1900 <= int(''.join(p)) <= 2024]
                if valid_years:
                    year = max(valid_years)
                    date_str = date_str.replace(year_match.group(0), str(year), 1)
                    return date_str
 
            elif year_length == 2:
                if year <= 24:  # Предполагаем, что 00-24 это 2000-2024
                    year_str = '20' + year_str
                else:  # Предполагаем, что 25-99 это 1925-1999
                    year_str = '19' + year_str
                year = int(year_str)
                date_str = date_str.replace(year_match.group(0), str(year), 1)
                return date_str
 
        # Если год не валиден, возвращаем измененную строку
        return date_str
 
    # Применяем обработку ко всей колонке
    return birthdate_series.apply(correct_date)
 
 
df['full_name'] = df['full_name'].apply(clean_fio)
df_3['name'] = df_3['name'].apply(clean_fio)
clean_and_merge_fio(df_2)
 
df['birthdate'] = process_birthdates(df['birthdate'])
df_2['birthdate'] = process_birthdates(df_2['birthdate'])
df_3['birthdate'] = process_birthdates(df_3['birthdate'])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.15 µs


In [40]:
# типа пайплайн маркировки

def clean_phone_number(phone):
    phone = re.sub(r'[^\d+]', '', phone)
    if phone.startswith('+'):
        phone = '+' + re.sub(r'[^\d]', '', phone[1:])
    else:
        phone = re.sub(r'[^\d]', '', phone)
    return phone
 
def random_noise_for_fio(full_name, tr = 0):
    def random_letter_modification(word):
        if random.random() < 0.5:
            if len(word) > 1:
                idx = random.randint(0, len(word) - 1)
                word = word[:idx] + word[idx+1:]
        else:
            idx = random.randint(0, len(word))
            word = word[:idx] + random.choice("АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ") + word[idx:]
        return word
 
    def random_word_swap(words):
        if len(words) > 1:
            idx1, idx2 = random.sample(range(len(words)), 2)
            words[idx1], words[idx2] = words[idx2], words[idx1]
        return words
 
    name_parts = full_name.split()
 
    if len(name_parts) > 2 and (random.random() < 0.1 or tr == 1):
        name_parts = name_parts[:2]
 
    name_parts = [random_letter_modification(part) for part in name_parts]
    
    if random.random() < 0.3:
        name_parts = random_word_swap(name_parts)
 
    noisy_name = " ".join(name_parts)
    return noisy_name
 
 
def random_noise_for_email(email):
    
    similar_letters = {'o': '0', 'l': '1', 'i': '1', 'e': '3', 'a': '4'}
    index = random.randint(0, len(email) - 2)
    if email[index] in similar_letters:
        email = email[:index] + similar_letters[email[index]] +  email[index + 1:]
 
    if random.random() < 1/3:
        if '@' in email:
            parts = email.split('@')
            email = parts[0]
            if random.choice([True, False]):
                email += '@' 
    
    return email
 
def random_noise_for_birthdate(birthdate):
    if random.choice([True, False]):
        year = int(birthdate[:4])
        new_year = year + random.randint(-1000, 1000)
        birthdate = str(new_year) + birthdate[4:]
    else:
        index = random.randint(0, len(birthdate) - 1)
        random_digit = random.choice(string.digits)
        birthdate =  birthdate[:index] + random_digit + birthdate[index + 1:]
    
    return birthdate
 
def random_noise_for_phone(phone):
    def random_digit_modification(number):
        if random.random() < 0.5:
            if len(number) > 1:
                idx = random.randint(0, len(number) - 1)
                number = number[:idx] + number[idx+1:]
        else:
            idx = random.randint(0, len(number))
            number = number[:idx] + str(random.randint(0, 9)) + number[idx:]
        return number
    phone = random_digit_modification(clean_phone_number(phone))
    
    return phone
 
def apply_noise(data, columns_to_noise, tr = 0):
    for column in columns_to_noise:
        try:
            if column in ['full_name', 'name']:
                data[column] = random_noise_for_fio(data[column], tr)
            elif column == 'email':
                data[column] = random_noise_for_email(data[column])
    
            elif column == 'birthdate':
                data[column] = random_noise_for_birthdate(data[column])
            elif column == 'phone':
                data[column] = random_noise_for_phone(data[column])
                
        except: pass
            
    return data
 
def apply_noise_to_date(dates):
    base_date = []
    new_date = []
    false_date = []
    
    for i in range(len(dates)):  
        row = dates.iloc[i] 
        row2 = dates.iloc[random.randint(1,len(dates)-1)] 
        r = random.random()
        r2 = random.random()
        r3 = random.random()
        
        if r < 1/3: 
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address', 'phone']], 1)
            apply_noise(row, columns_to_noise, tr=1)  
            new_date.append(f"{row['full_name']}, {row['email']}, {row['sex']}, {row['birthdate']}, 00000000, ")
        elif r < 2/3:
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address', 'email', 'sex']], 1)
            apply_noise(row, columns_to_noise)  
            new_date.append(f"{row['full_name']}, 00000000, 00000000, {row['birthdate']}, {row['phone']}")
        else:
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address']], 1)
            apply_noise(row, columns_to_noise)  
            new_date.append(f"{row['full_name']}, {row['email']}, {row['sex']}, {row['birthdate']}, {row['phone']}")
            
        if r2 < 1/3: 
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address', 'phone']], 1)
            apply_noise(row2, columns_to_noise, tr=1)  
            false_date.append(f"{row2['full_name']}, {row2['email']}, {row2['sex']}, {row2['birthdate']}, 00000000, ")
        elif r2 < 2/3:
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address', 'email', 'sex']], 1)
            apply_noise(row2, columns_to_noise)  
            false_date.append(f"{row2['full_name']}, 00000000, 00000000, {row2['birthdate']}, {row2['phone']}")
        else:
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address']], 1)
            apply_noise(row2, columns_to_noise)  
            false_date.append(f"{row2['full_name']}, {row2['email']}, {row2['sex']}, {row2['birthdate']}, {row2['phone']}")
 
        if r3 < 1/3: 
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address', 'phone']], 1)
            apply_noise(row, columns_to_noise, tr=1)  
            base_date.append(f"{row['full_name']}, {row['email']}, {row['sex']}, {row['birthdate']}, 00000000, ")
        elif r3 < 2/3:
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address', 'email', 'sex']], 1)
            apply_noise(row, columns_to_noise)  
            base_date.append(f"{row['full_name']}, 00000000, 00000000, {row['birthdate']}, {row['phone']}")
        else:
            columns_to_noise = random.sample([col for col in data.columns if col not in ['uid', 'address']], 1)
            apply_noise(row, columns_to_noise)  
            base_date.append(f"{row['full_name']}, {row['email']}, {row['sex']}, {row['birthdate']}, {row['phone']}")
    
    return base_date, new_date, false_date
        
 
data = df.head(100000).copy()
 
# base_date, new_data, false_date = apply_noise_to_date(data)
dataset = apply_noise_to_date(data)

In [38]:
token_dict = {
    '$':  0.0,
    '/':  40.0,
    '+':  39.0,
    '(':  38.0,
    ')':  37.0,
    ',':  36.0,
    ' ':  35.0,
    '_':  34.0,
    'о':  33.0,
    'е':  32.0,
    'ё':  31.0,
    'а':  30.0,
    'и':  29.0,
    'н':  28.0,
    'т':  27.0,
    'с':  26.0,
    'р':  25.0,
    'в':  24.0,
    'л':  23.0,
    'к':  22.0,
    'м':  21.0,
    'д':  20.0,
    'п':  19.0,
    'у':  18.0,
    'я':  17.0,
    'з':  16.0,
    'ы':  15.0,
    'б':  14.0,
    'ь':  13.0,
    'ъ':  12.0,
    'г':  11.0,
    'ч':  10.0,
    'й':  9.0,
    'х':  8.0,
    'ж':  7.0,
    'ш':  6.0,
    'ю':  5.0,
    'ц':  4.0,
    'щ':  3.0,
    'э':  2.0,
    'ф':  1.0,
    '.':  -41.0,
    '0':  -40.0,
    '1':  -39.0,
    '2':  -38.0,
    '3':  -37.0,
    '4':  -36.0,
    '5':  -35.0,
    '6':  -34.0,
    '7':  -33.0,
    '8':  -32.0,
    '9':  -31.0,
    '"':  -30.0,
    '-':  -29.0,
    ';':  -28.0,
    '@':  -27.0,
    'e':  -26.0,
    't':  -25.0,
    'a':  -24.0,
    'o':  -23.0,
    'n':  -22.0,
    'i':  -21.0,
    's':  -20.0,
    'r':  -19.0,
    'h':  -18.0,
    'l':  -17.0,
    'd':  -16.0,
    'c':  -15.0,
    'u':  -14.0,
    'p':  -13.0,
    'f':  -12.0,
    'm':  -11.0,
    'w':  -10.0,
    'y':  -9.0,
    'b':  -8.0,
    'g':  -7.0,
    'v':  -6.0,
    'k':  -5.0,
    'q':  -4.0,
    'x':  -3.0,
    'j':  -2.0,
    'z':  -1.0
}

def tokenizer(S):
    if len(S)<64:
        S+="$"*(64-len(S))
    return torch.tensor([token_dict[i] for i in S.lower()]).unsqueeze(0).view((-1,1))/41.0

In [6]:
# эталонный размер тензоров похожести
# АХТУНГ: исходные тензоры ОБЯЗАНЫ иметь размер НЕ МЕНЬШИЙ эталонного
Tensor_Standart = (64,64)

class SimilarityTensor(nn.Module):
    def __init__(self):
        super(SimilarityTensor, self).__init__()

        self.flatten = nn.Flatten()
        
        self.conv1 = nn.Sequential(
            nn.Linear(in_features = 1, out_features = 16),
            nn.LeakyReLU(),
            nn.Linear(in_features = 16, out_features = 32),
            nn.LeakyReLU(),
            nn.Linear(in_features = 32, out_features = 32),
            nn.LeakyReLU(),
            nn.Linear(in_features = 32, out_features = 16)
        )
        

        self.conv2 = nn.Sequential(
            nn.Linear(in_features = 16, out_features = 32),
            nn.LeakyReLU(),
            nn.Linear(in_features = 32, out_features = 64),
            nn.LeakyReLU(),
            nn.Linear(in_features = 64, out_features = 64),
            nn.LeakyReLU(),
            nn.Linear(in_features = 64, out_features = 16)
        )


        self.conv3 = nn.Sequential(
            nn.Linear(in_features = 16, out_features = 32),
            nn.LeakyReLU(),
            nn.Linear(in_features = 32, out_features = 64),
            nn.LeakyReLU(),
            nn.Linear(in_features = 64, out_features = 64),
            nn.LeakyReLU(),
            nn.Linear(in_features = 64, out_features = 16) 
        )


        self.conv4 = nn.Sequential(
            nn.Linear(in_features = 16, out_features = 32),
            nn.LeakyReLU(),
            nn.Linear(in_features = 32, out_features = 64),
            nn.LeakyReLU(),
            nn.Linear(in_features = 64, out_features = 64),
            nn.LeakyReLU(),
            nn.Linear(in_features = 64, out_features = 16) 
        )

        self.FC1 = nn.Sequential(
            nn.Conv2d(in_channels = 4, out_channels = 32, kernel_size = 4, stride = 2, padding=0, bias = True),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 4, stride = 2, padding=0, bias = True),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 4, stride = 2, padding=0, bias = True),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels = 64, out_channels = 48, kernel_size = 4, stride = 2, padding=0, bias = True),
            nn.LeakyReLU()
        )
        
        self.FC2 = nn.Sequential(
            nn.Linear(in_features = 192, out_features = 400),
            nn.LeakyReLU(),
            nn.Linear(in_features = 400, out_features = 200),
            nn.LeakyReLU(),
            nn.Linear(in_features = 200, out_features = 1),
            nn.Sigmoid()
        )

    # получение пуллинга приводящего тезор к эталонному формату по осям -1 и -2
    def get_pool(self, cur_shape:tuple, target_shape: tuple):
        if cur_shape[-1] < target_shape[-1] or cur_shape[-2] < target_shape[-2]:
            raise ValueError('target shape mast be smaller then current')
    
        kernel_size = (cur_shape[-2]%(target_shape[-2]-1),cur_shape[-1]%(target_shape[-1]-1))
        stride = (cur_shape[-2]//(target_shape[-2]-1),cur_shape[-1]//(target_shape[-1]-1))
        return nn.AvgPool2d(kernel_size, stride)

    def forward(self, S_b, S_t, S_f):
        # Создаем тензоры похожести
        #--- 1 уровнь эмбедингов
        S_b = self.conv1(S_b)
        S_t = self.conv1(S_t)
        S_f = self.conv1(S_f)

        # создаем матрицы со скалярными произведениями каждой пары эмбедингов
        BT_embeding = torch.matmul(S_t.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))
        BF_embeding = torch.matmul(S_f.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))

        # АХТУНГ: инплейс операция
        # Добавляем матрицам похожести новое измерение для сложения единого тензора из них
        BT_embeding.unsqueeze_(0)
        BF_embeding.unsqueeze_(0)

        #--- 2 уровнь эмбедингов
        S_b = self.conv2(S_b)
        S_t = self.conv2(S_t)
        S_f = self.conv2(S_f)

        # матрицы похожести 2 уровня эмбедингов
        BT_embeding_onstep = torch.matmul(S_t.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))
        BF_embeding_onstep = torch.matmul(S_f.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))

        BT_embeding_onstep.unsqueeze_(0)
        BF_embeding_onstep.unsqueeze_(0)

        # соединяем матрицы похожести
        BT_embeding = torch.cat((BT_embeding,BT_embeding_onstep),0)
        BF_embeding = torch.cat((BF_embeding,BF_embeding_onstep),0)
        
        #--- 3 уровнь эмбедингов
        S_b = self.conv3(S_b)
        S_t = self.conv3(S_t)
        S_f = self.conv3(S_f)
        
        # матрицы похожести 3 уровня эмбедингов
        BT_embeding_onstep = torch.matmul(S_t.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))
        BF_embeding_onstep = torch.matmul(S_f.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))

        BT_embeding_onstep.unsqueeze_(0)
        BF_embeding_onstep.unsqueeze_(0)

        BT_embeding = torch.cat((BT_embeding,BT_embeding_onstep),0)
        BF_embeding = torch.cat((BF_embeding,BF_embeding_onstep),0)

        #--- 4 уровнь эмбедингов
        S_b = self.conv4(S_b)
        S_t = self.conv4(S_t)
        S_f = self.conv4(S_f)

        BT_embeding_onstep = torch.matmul(S_t.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))
        BF_embeding_onstep = torch.matmul(S_f.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))

        BT_embeding_onstep.unsqueeze_(0)
        BF_embeding_onstep.unsqueeze_(0)

        BT_embeding = torch.cat((BT_embeding,BT_embeding_onstep),0)
        BF_embeding = torch.cat((BF_embeding,BF_embeding_onstep),0)
        
        # На основании тензоров похожести ищем ими описанное расстояние
        # Приводим тензоры к эталонному размеру
        BT_pool = self.get_pool(BT_embeding.shape, Tensor_Standart)
        BF_pool = self.get_pool(BF_embeding.shape, Tensor_Standart)

        BT_embeding = BT_pool(BT_embeding)
        BF_embeding = BF_pool(BF_embeding)

        # пропускаем тензоры 4х64х64 через свертки
        # для сверточного слоя тензор должен иметь 4D shape, приводим к (1,4,64,64)
        BT_embeding.unsqueeze_(0)
        BF_embeding.unsqueeze_(0)
        
        BT_embeding = self.FC1(BT_embeding)
        BF_embeding = self.FC1(BF_embeding)

        # линеаризуем результат
        BT_embeding = self.flatten(BT_embeding)
        BF_embeding = self.flatten(BF_embeding)

        # извлекаем меру похожести между BT и BF
        BT_embeding = self.FC2(BT_embeding)
        BF_embeding = self.FC2(BF_embeding)

        return BT_embeding, BF_embeding

    def get_dis(self, S_b, S_t):
        #--- 1 уровнь эмбедингов
        S_b = self.conv1(S_b)
        S_t = self.conv1(S_t)

        # создаем матрицу со скалярными произведениями каждой пары эмбедингов
        BT_embeding = torch.matmul(S_t.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))

        # Добавляем матрице похожести новое измерение для сложения единого тензора
        BT_embeding.unsqueeze_(0)

        #--- 2 уровнь эмбедингов
        S_b = self.conv2(S_b)
        S_t = self.conv2(S_t)

        # матрицы похожести 2 уровня эмбедингов
        BT_embeding_onstep = torch.matmul(S_t.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))

        BT_embeding_onstep.unsqueeze_(0)

        # соединяем матрицы похожести
        BT_embeding = torch.cat((BT_embeding,BT_embeding_onstep),0)

        #--- 3 уровнь эмбедингов
        S_b = self.conv3(S_b)
        S_t = self.conv3(S_t)

        # матрицы похожести 3 уровня эмбедингов
        BT_embeding_onstep = torch.matmul(S_t.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))
        
        BT_embeding_onstep.unsqueeze_(0)

        BT_embeding = torch.cat((BT_embeding,BT_embeding_onstep),0)

        #--- 4 уровнь эмбедингов
        S_b = self.conv4(S_b)
        S_t = self.conv4(S_t)

        BT_embeding_onstep = torch.matmul(S_t.view((-1,16)), torch.transpose(S_b.view((-1,16)), -2,-1))
        
        BT_embeding_onstep.unsqueeze_(0)

        BT_embeding = torch.cat((BT_embeding,BT_embeding_onstep),0)

        # На основании тензоров похожести ищем ими описанное расстояние
        # Приводим тензоры к эталонному размеру
        BT_pool = self.get_pool(BT_embeding.shape, Tensor_Standart)

        BT_embeding = BT_pool(BT_embeding)

        # пропускаем тензоры 4х64х64 через свертки
        # для сверточного слоя тензор должен иметь 4D shape, приводим к (1,4,64,64)
        BT_embeding.unsqueeze_(0)

        BT_embeding = self.FC1(BT_embeding)

        # линеаризуем результат
        BT_embeding = self.flatten(BT_embeding)

        # извлекаем меру похожести между BT и BF
        BT_embeding = self.FC2(BT_embeding)

        return BT_embeding

In [7]:
alpha_margin = torch.tensor([0.1])
def criterion(BT_embeding, BF_embeding):
    return torch.max(BT_embeding - BF_embeding + alpha_margin, torch.zeros((1,1)))

In [8]:
B = 'БАЛКИБАЕВ МАМАСАИД КАРЛИТО,mamasaid_balkibaev0@example.ru,m,1954-06-15,0206450526'
T = 'АЛКИБАЕВ МАМАСАИД КАРЛИТО,mamasaid_balkibaev0@example.ru,m,2954-06-15,0206450526'
F = 'ИГИНОВА ЮТТА КОНДРАТЬЕВНП,jutta_iginova1@yandex.ru,f,1981-11-24,1971286327'

In [36]:
model = SimilarityTensor()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.003)
data_loader = dataset

train_loss = []
val_loss   = []

summary(model, [(100,1,1),(100,1,1),(100,1,1)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1           [-1, 100, 1, 16]              32
         LeakyReLU-2           [-1, 100, 1, 16]               0
            Linear-3           [-1, 100, 1, 32]             544
         LeakyReLU-4           [-1, 100, 1, 32]               0
            Linear-5           [-1, 100, 1, 32]           1,056
         LeakyReLU-6           [-1, 100, 1, 32]               0
            Linear-7           [-1, 100, 1, 16]             528
            Linear-8           [-1, 100, 1, 16]              32
         LeakyReLU-9           [-1, 100, 1, 16]               0
           Linear-10           [-1, 100, 1, 32]             544
        LeakyReLU-11           [-1, 100, 1, 32]               0
           Linear-12           [-1, 100, 1, 32]           1,056
        LeakyReLU-13           [-1, 100, 1, 32]               0
           Linear-14           [-1, 100

In [41]:
n_epoch = 1

for epoch in tqdm(range(n_epoch)):
    model.train()
    train_loss_per_epoch =[]
    for i in range(len(data_loader[0])):
        optimizer.zero_grad()
        BT_dif, BF_dif = model(tokenizer(data_loader[0][i]),tokenizer(data_loader[1][i]),tokenizer(data_loader[2][i]))
        loss = criterion(BT_dif, BF_dif)
        loss.backward()
        optimizer.step()
        train_loss_per_epoch.append(loss.item())
        
    train_loss.append(np.mean(train_loss_per_epoch))

  0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
len(data_loader[0])

500